In [1]:
import psycopg2
import json
import requests
import os
import datetime
import pandas as pd
from psycopg2 import OperationalError

In [9]:
#Aux functs

def read_folder_file(folder_name, file_name):
    dir_path = os.path.dirname(os.getcwd())
    folders = [folder for folder in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, folder))]

    for folder in folders:
        if folder == folder_name:
            folder_path = os.path.join(dir_path, folder)
            files = [file for file in os.listdir(folder_path)]

            for file in files:
                if file==file_name:
                    return os.path.join(folder_path,file_name)

    return None

def get_credentials(service_name,json_path):
    with open(json_path, "r") as file:
        data = json.load(file)

    passwords = data.get("passwords", [])

    for entry in passwords:
        if entry["service"] == service_name:
            return entry
    return None

def run_pg_qry(credentials, qry):
    conn = None
    cursor = None
    try:
        conn = psycopg2.connect(
            host=credentials['hostname'],
            database=credentials['database'],
            port=credentials['port'],
            user=credentials['username'],
            password=credentials['password']
        )
        
        cursor = conn.cursor()        
        cursor.execute(qry)
        rows = cursor.fetchall()
        
        conn.commit()

    except (Exception, OperationalError) as error:
        print(f"Error ejecutando la consulta: {error}")
        if conn:
            conn.rollback()

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

    return rows
    
def insert_to_postgres(credentials,data,table_name):

    conn = psycopg2.connect(
        host=credentials['hostname'],
        database=credentials['database'],
        port=credentials['port'],
        user=credentials['username'],
        password=credentials['password']
    )

    date_uploaded = datetime.datetime.now()
    uploaded_by = credentials['username']

    try:
        cursor = conn.cursor()
        
        insert_query = f"""
        INSERT INTO {table_name} ("date", "open", high, low, "close", date_uploaded, uploaded_by)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """
        
        count = 0

        for entry in data:
            timestamp = pd.to_datetime(entry[0], unit='ms')
            open_price = float(entry[1])
            high_price = float(entry[2])
            low_price = float(entry[3])
            close_price = float(entry[4])

            cursor.execute(insert_query, 
                           (timestamp, open_price, high_price, low_price, close_price, date_uploaded, uploaded_by))
            count+=1

        conn.commit()
        print(f"Filas insertadas: {count}")

    except psycopg2.Error as insert_error:
        print("Error al insertar datos:", insert_error)
        conn.rollback()

    except Exception as e:
        print(f"Se produjo error inesperado: {e}")
        conn.rollback()
    
    finally:  
        cursor.close()
        conn.close()


def convert_to_milliseconds(date_str):
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int(date.timestamp() * 1000)


def get_binance_ohlc(start_date, end_date, symbol="BTCUSDT", interval="1m"):
    url = "https://api.binance.com/api/v3/klines"
    
    start_time = convert_to_milliseconds(start_date)
    end_time = convert_to_milliseconds(end_date)
    
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time,
        "limit":1000
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    return data

In [3]:
#SET params
file_name = 'keys.json'
folder_name = 'keys'
service_name = "pgcurrencies"

json_path = read_folder_file(folder_name, file_name)
credentials = get_credentials(service_name,json_path)

In [4]:



start_date = "2024-01-01 00:00:00"
end_date = "2024-01-02 00:00:00"
table_name = 'public.binance_btcusd'
data = get_binance_ohlc(start_date, end_date)

insert_to_postgres(credentials,data,table_name)


Filas insertadas: 1000


In [13]:
df = pd.DataFrame(data, columns=["date", "open", "high", "low", "close", "volume", "_", "_", "_", "_", "_", "_"])
df = df[["date", "open", "high", "low", "close"]]
df["date"] = pd.to_datetime(df["date"], unit="ms")

df

,date,open,high,low,close
0,2024-01-01 00:00:00,42283.58000000,42298.62000000,42261.02000000,42298.61000000
1,2024-01-01 00:01:00,42298.62000000,42320.00000000,42298.61000000,42320.00000000
2,2024-01-01 00:02:00,42319.99000000,42331.54000000,42319.99000000,42325.50000000
3,2024-01-01 00:03:00,42325.50000000,42368.00000000,42325.49000000,42367.99000000
4,2024-01-01 00:04:00,42368.00000000,42397.23000000,42367.99000000,42397.23000000
...,...,...,...,...,...
995,2024-01-01 16:35:00,42727.68000000,42738.00000000,42727.67000000,42738.00000000
996,2024-01-01 16:36:00,42738.00000000,42748.00000000,42737.99000000,42747.99000000
997,2024-01-01 16:37:00,42748.00000000,42748.00000000,42723.07000000,42724.65000000
998,2024-01-01 16:38:00,42724.65000000,42768.30000000,42724.65000000,42760.25000000


In [12]:
qry='''
SELECT MAX(date) FROM binance_btcusd
'''


run_pg_qry(credentials,qry)[0][0]

datetime.datetime(2024, 1, 1, 16, 39)